<a href="https://colab.research.google.com/github/jfreedman340/Financial-Markets/blob/main/param_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install backtrader

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install datetime

In [ ]:
pip install qgrid

In [ ]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import matplotlib
from datetime import datetime
import qgrid

In [ ]:
class MaCrossStrategy(bt.Strategy):
 
    params = (
        ('fast_length', 10),
        ('slow_length', 50)
    )
     
    def __init__(self):
        ma_fast = bt.ind.SMA(period = self.params.fast_length)
        ma_slow = bt.ind.SMA(period = self.params.slow_length)
         
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)
 
    def next(self):
        if not self.position:
            if self.crossover > 0: 
                self.buy()
        elif self.crossover < 0: 
            self.close()

In [ ]:
cerebro = bt.Cerebro()
 
data = bt.feeds.YahooFinanceCSVData(dataname = 'AAPL.csv', fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
cerebro.adddata(data)
 
strats = cerebro.optstrategy(
        MaCrossStrategy,
        fast_length = range(1, 10), 
        slow_length = range(1, 10))
 
 
cerebro.broker.setcash(1000000.0)
 
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.DrawDown, _name = "drawdown")
cerebro.addanalyzer(btanalyzers.Returns, _name = "returns")
 
back = cerebro.run(maxcpus=1)

In [ ]:
back

In [ ]:
back[0][0].analyzers.sharpe.get_analysis()['sharperatio']

In [ ]:
back[0][0].analyzers.drawdown.get_analysis()['max']['drawdown']

0.0

In [ ]:
back[0][0].analyzers.returns.get_analysis()['rnorm100']

0.0

In [ ]:
par_list = [[x[0].params.fast_length, 
             x[0].params.slow_length,
             x[0].analyzers.returns.get_analysis()['rnorm100'], 
             x[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
             x[0].analyzers.sharpe.get_analysis()['sharperatio']
            ] for x in back]
par_list

[[1, 1, 0.0, 0.0, None],
 [1, 2, 1.0256488374612283, 4.034631254120626, -0.07010469200326193],
 [1, 3, 0.5704335825722359, 3.783892989231391, -0.3868507681478207],
 [1, 4, 0.7522490093584738, 4.384315654607195, -0.21949785758301824],
 [1, 5, 1.146470688878326, 4.166733729779052, -0.045516335624524354],
 [1, 6, 1.8856948864193466, 3.318668561755501, 0.5070910577082068],
 [1, 7, 2.23779941804275, 2.9984331555977697, 0.7961543275757154],
 [1, 8, 2.503403810936046, 2.764329796924742, 0.696392880184903],
 [1, 9, 2.4840633320203187, 2.6360273930559286, 0.6539034736425332],
 [2, 1, 2.1730826175728692, 1.7613729258807906, 0.7659435778882535],
 [2, 2, 0.0, 0.0, None],
 [2, 3, 0.5628199642525639, 3.6973021256558467, -0.3763613344476699],
 [2, 4, 1.2333998273677607, 4.155073523190263, 0.018786933186745118],
 [2, 5, 2.1519093878983675, 3.8053058585190715, 1.0352725159806289],
 [2, 6, 2.257352723363694, 3.2146756632624576, 0.6348933371268533],
 [2, 7, 2.665622995295057, 2.2120342253052345, 0.718949

In [ ]:
par_df = pd.DataFrame(par_list, columns = ['length_fast', 'length_slow', 'return', 'dd', 'sharpe'])

In [ ]:
par_df

,length_fast,length_slow,return,dd,sharpe
0,1,1,0.000000,0.000000,NaN
1,1,2,1.025649,4.034631,-0.070105
2,1,3,0.570434,3.783893,-0.386851
3,1,4,0.752249,4.384316,-0.219498
4,1,5,1.146471,4.166734,-0.045516
...,...,...,...,...,...
76,9,5,-0.216719,3.477712,-1.470182
77,9,6,-0.792021,3.761731,-1.639549
78,9,7,-0.129517,3.669453,-1.116663
79,9,8,0.794821,3.824403,-0.276751


In [ ]:
qgrid.show_grid(par_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…